In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler(feature_range=(0, 1), copy=True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error # 平均二乗誤差
#from sklearn.metrics import mean_squared_log_error # 対数平均二乗誤差
from sklearn.metrics import r2_score # 決定係数
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import missingno as msno
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

# 1.Import data

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/test.csv")

In [ ]:
# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.info()

In [ ]:
train

In [ ]:
train.describe()

In [ ]:
test

In [ ]:
test.describe()

# 2.EDA

In [ ]:
# Colors to be used for plots
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen", "lightseagreen",
          "cornflowerblue", "mediumpurple", "palevioletred", "lightskyblue", "sandybrown",
          "yellowgreen", "indianred", "lightsteelblue", "mediumorchid", "deepskyblue"]

In [ ]:
# Search for missing data

msno.matrix(df=train, figsize=(10,6), color=(0,.3,.3))

In [ ]:
msno.matrix(df=test, figsize=(10,6), color=(0,.3,.3))

In [ ]:
# Concat train and test
all = pd.concat([train,test],ignore_index=True)


In [ ]:

all.drop(columns=['row_id', 'target']).describe().T\
        .style.bar(subset=['mean'], color=px.colors.qualitative.G10[0])\
        .background_gradient(subset=['std'], cmap='Greens')\
        .background_gradient(subset=['50%'], cmap='BuGn')


In [ ]:

%%time
corr = train.drop(columns=["row_id"]).corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr, vmin=-1, vmax=1, center=0, square=False, annot=False, cmap='coolwarm')
plt.show()


In [ ]:

%%time
corr = test.drop(columns=["row_id"]).corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr, vmin=-1, vmax=1, center=0, square=False, annot=False, cmap='coolwarm')
plt.show()


In [ ]:
train["target"].value_counts()

In [ ]:

fig, ax = plt.subplots(figsize=(16, 8))

bars = ax.bar(train["target"]
              .value_counts()
#              .sort_index()
              .index,
              train["target"]
              .value_counts()
#              .sort_index()
              .values,
              color=colors,
              edgecolor="black")
ax.set_title("Target distribution", fontsize=20, pad=15)
ax.set_ylabel("Count", fontsize=14, labelpad=15)
ax.set_xlabel("Target value", fontsize=14, labelpad=10)
ax.bar_label(bars, [f"{x:2.2f}%" for x in train["target"]
                    .value_counts()
#                    .sort_index()
                    .values/(len(train)/100)],
                 padding=5, fontsize=10, rotation=90)
ax.margins(0.025, 0.12)
ax.grid(axis="y")

plt.show();


In [ ]:

%%time
df = pd.concat([train.drop(["row_id", "target"], axis=1), test.drop("row_id", axis=1)], axis=0)
columns = df.columns.values

cols = 3
rows = len(columns) // cols + 1

fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(15,150), sharex=False)

plt.subplots_adjust(hspace = 1.5)
i=0

for r in np.arange(0, rows, 1):
    for c in np.arange(0, cols, 1):
        if i >= len(columns):
            axs[r, c].set_visible(False)
        else:
            hist1 = axs[r, c].hist(train[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="cornflowerblue",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Train")
            hist2 = axs[r, c].hist(test[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="darkorange",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Test")
            axs[r, c].set_title(columns[i], fontsize=14, pad=5)
            axs[r, c].tick_params(axis="y", labelsize=13)
            axs[r, c].tick_params(axis="x", labelsize=13)
            axs[r, c].grid(axis="y")
            axs[r, c].legend(fontsize=13)
                                  
        i+=1

plt.show();


# 3.Preprocessing

In [ ]:

# Add rare cases as features.
# Search for 10 consecutive sequences that can have the same base sequence.
# And search for sequences in which 9 of the 10 bases are the same.

ary_A10_train = []
ary_A10_test = []
ary_T10_train = []
ary_T10_test = []
ary_G10_train = []
ary_G10_test = []
ary_C10_train = []
ary_C10_test = []
ary_A9_train = []
ary_A9_test = []
ary_T9_train = []
ary_T9_test = []
ary_G9_train = []
ary_G9_test = []
ary_C9_train = []
ary_C9_test = []
for i in range(len(train['row_id'])):
    if train['A10T0G0C0'][i] > 0:
        ary_A10_train.append(train['A10T0G0C0'][i])
    else:
        ary_A10_train.append(0)
    if train['A0T10G0C0'][i] > 0:
        ary_T10_train.append(train['A0T10G0C0'][i])
    else:
        ary_T10_train.append(0)
    if train['A0T0G10C0'][i] > 0:
        ary_G10_train.append(train['A0T0G10C0'][i])
    else:
        ary_G10_train.append(0)
    if train['A0T0G0C10'][i] > 0:
        ary_C10_train.append(train['A0T0G0C10'][i])
    else:
        ary_C10_train.append(0)
    if train['A9T1G0C0'][i] > 0 or train['A9T0G1C0'][i] > 0 or train['A9T0G0C1'][i] > 0:
        if train['A9T1G0C0'][i] > 0:
            A_1 = train['A9T1G0C0'][i]
        else:
            A_1 = 0
        if train['A9T0G1C0'][i] > 0:
            A_2 = train['A9T0G1C0'][i]
        else:
            A_2 = 0
        if train['A9T0G0C1'][i] > 0:
            A_3 = train['A9T0G0C1'][i]
        else:
            A_3 = 0
        ary_A9_train.append(A_1 + A_2 + A_3)
    else:
        ary_A9_train.append(0)
    if train['A1T9G0C0'][i] > 0 or train['A0T9G1C0'][i] > 0 or train['A0T9G0C1'][i] > 0:
        if train['A1T9G0C0'][i] > 0:
            T_1 = train['A1T9G0C0'][i]
        else:
            T_1 = 0
        if train['A0T9G1C0'][i] > 0:
            T_2 = train['A0T9G1C0'][i]
        else:
            T_2 = 0
        if train['A0T9G0C1'][i] > 0:
            T_3 = train['A0T9G0C1'][i]
        else:
            T_3 = 0
        ary_T9_train.append(T_1 + T_2 + T_3)
    else:
        ary_T9_train.append(0)
    if train['A1T0G9C0'][i] > 0 or train['A0T1G9C0'][i] > 0 or train['A0T0G9C1'][i] > 0:
        if train['A1T0G9C0'][i] > 0:
            G_1 = train['A1T0G9C0'][i]
        else:
            G_1 = 0
        if train['A0T1G9C0'][i] > 0:
            G_2 = train['A0T1G9C0'][i]
        else:
            G_2 = 0
        if train['A0T0G9C1'][i] > 0:
            G_3 = train['A0T0G9C1'][i]
        else:
            G_3 = 0
        ary_G9_train.append(G_1 + G_2 + G_3)
    else:
        ary_G9_train.append(0)
    if train['A1T0G0C9'][i] > 0 or train['A0T1G0C9'][i] > 0 or train['A0T0G1C9'][i] > 0:
        if train['A1T0G0C9'][i] > 0:
            C_1 = train['A1T0G0C9'][i]
        else:
            C_1 = 0
        if train['A0T1G0C9'][i] > 0:
            C_2 = train['A0T1G0C9'][i]
        else:
            C_2 = 0
        if train['A0T0G9C1'][i] > 0:
            C_3 = train['A0T0G9C1'][i]
        else:
            C_3 = 0
        ary_C9_train.append(C_1 + C_2 + C_3)
    else:
        ary_C9_train.append(0)
    
for i in range(len(test['row_id'])):
    if test['A10T0G0C0'][i] > 0:
        ary_A10_test.append(test['A10T0G0C0'][i])
    else:
        ary_A10_test.append(0)
    if test['A0T10G0C0'][i] > 0:
        ary_T10_test.append(test['A0T10G0C0'][i])
    else:
        ary_T10_test.append(0)
    
    if test['A0T0G10C0'][i] > 0:
        ary_G10_test.append(test['A0T0G10C0'][i])
    else:
        ary_G10_test.append(0)
    if test['A0T0G0C10'][i] > 0:
        ary_C10_test.append(test['A0T0G0C10'][i])
    else:
        ary_C10_test.append(0)
    if test['A9T1G0C0'][i] > 0 or test['A9T0G1C0'][i] > 0 or test['A9T0G0C1'][i] > 0:
        if test['A9T1G0C0'][i] > 0:
            A_1 = test['A9T1G0C0'][i]
        else:
            A_1 = 0
        if test['A9T0G1C0'][i] > 0:
            A_2 = test['A9T0G1C0'][i]
        else:
            A_2 = 0
        if test['A9T0G0C1'][i] > 0:
            A_3 = test['A9T0G0C1'][i]
        else:
            A_3 = 0
        ary_A9_test.append(A_1 + A_2 + A_3)
    else:
        ary_A9_test.append(0)
    if test['A1T9G0C0'][i] > 0 or test['A0T9G1C0'][i] > 0 or test['A0T9G0C1'][i] > 0:
        if test['A1T9G0C0'][i] > 0:
            T_1 = test['A1T9G0C0'][i]
        else:
            T_1 = 0
        if test['A0T9G1C0'][i] > 0:
            T_2 = test['A0T9G1C0'][i]
        else:
            T_2 = 0
        if test['A0T9G0C1'][i] > 0:
            T_3 = test['A0T9G0C1'][i]
        else:
            T_3 = 0
        ary_T9_test.append(T_1 + T_2 + T_3)
    else:
        ary_T9_test.append(0)
    if test['A1T0G9C0'][i] > 0 or test['A0T1G9C0'][i] > 0 or test['A0T0G9C1'][i] > 0:
        if test['A1T0G9C0'][i] > 0:
            G_1 = test['A1T0G9C0'][i]
        else:
            G_1 = 0
        if test['A0T1G9C0'][i] > 0:
            G_2 = test['A0T1G9C0'][i]
        else:
            G_2 = 0
        if test['A0T0G9C1'][i] > 0:
            G_3 = test['A0T0G9C1'][i]
        else:
            G_3 = 0
        ary_G9_test.append(G_1 + G_2 + G_3)
    else:
        ary_G9_test.append(0)
    if test['A1T0G0C9'][i] > 0 or test['A0T1G0C9'][i] > 0 or test['A0T0G1C9'][i] > 0:
        if test['A1T0G0C9'][i] > 0:
            C_1 = test['A1T0G0C9'][i]
        else:
            C_1 = 0
        if test['A0T1G0C9'][i] > 0:
            C_2 = test['A0T1G0C9'][i]
        else:
            C_2 = 0
        if test['A0T0G1C9'][i] > 0:
            C_3 = test['A0T0G1C9'][i]
        else:
            C_3 = 0
        ary_C9_test.append(C_1 + C_2 + C_3)
    else:
        ary_C9_test.append(0)

train['isA10'] = ary_A10_train
test['isA10'] = ary_A10_test
train['isT10'] = ary_T10_train
test['isT10'] = ary_T10_test
train['isG10'] = ary_G10_train
test['isG10'] = ary_G10_test
train['isC10'] = ary_C10_train
test['isC10'] = ary_C10_test
train['isA9'] = ary_A9_train
test['isA9'] = ary_A9_test
train['isT9'] = ary_T9_train
test['isT9'] = ary_T9_test
train['isG9'] = ary_G9_train
test['isG9'] = ary_G9_test
train['isC9'] = ary_C9_train
test['isC9'] = ary_C9_test


In [ ]:
# Concat train and test (2)
all_2 = pd.concat([train,test],ignore_index=True)
all_2

In [ ]:
columns = all_2.columns.drop(['row_id', 'target'])
row_id = all_2['row_id']
target = all_2['target']

In [ ]:
columns

In [ ]:
row_id

In [ ]:
target

In [ ]:
# Scaling

np_all_2 = np.array(all_2.drop(columns=['row_id', 'target']))
data_one_column = np_all_2.reshape([-1,1])
data_one_column_scale = mmscaler.fit_transform(data_one_column)
all_2_scale = data_one_column_scale.reshape(np_all_2.shape)
all_2_scale

In [ ]:

all_2_df = pd.DataFrame(all_2_scale, columns=columns)
all_2_df['row_id'] = row_id
all_2_df['target'] = target


In [ ]:
# Split all for train and test
train_scale = all_2_df.iloc[train.index[0]:train.index[-1]+1]
test_scale = all_2_df.iloc[train.index[-1]+1:]
test_scale = test_scale.drop(columns=["target"])

In [ ]:
train_scale

In [ ]:
test_scale

# 4.Modeling

In [ ]:
train_scale['target'] = le.fit_transform(train_scale['target'])
train_scale

In [ ]:
X = train_scale.drop(columns=["row_id", "target"])
value = train_scale["target"]

In [ ]:
%%time

X_train, X_test, t_train, t_test = train_test_split(X, value, test_size=0.2, random_state=0)

lgb_train = lgb.Dataset(X_train, t_train)
lgb_eval = lgb.Dataset(X_test, t_test, reference=lgb_train)

params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 10,
        'learning_rate': 0.4,# 0.5
        'num_leaves': 3,
        'max_bin': 234,#234
        'num_iterations': 20000,
        'verbosity': -1
}

model = lgb.train(
    params,
    train_set=lgb_train,
    valid_sets=lgb_eval,
    early_stopping_rounds=100,
    verbose_eval=100
)

# 5.Prediction

In [ ]:
%%time

X_test = test_scale.drop(columns=["row_id"])

pred_array = model.predict(X_test)
sample_submission['target'] = np.argmax(pred_array, axis=1) # 最尤と判断したクラスの値にする


In [ ]:
sample_submission['target'] = le.inverse_transform(sample_submission['target'])
sample_submission

# 6.Make submission file

In [ ]:
sample_submission.to_csv('submission.csv', index=False)